In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, desc, sum, lag
from pyspark.sql.types import DateType
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('Stock_Analysis').getOrCreate()

23/07/07 16:12:20 WARN Utils: Your hostname, all-MS-7D35 resolves to a loopback address: 127.0.1.1; using 192.168.1.108 instead (on interface enp2s0)
23/07/07 16:12:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/07 16:12:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/07 16:12:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [4]:
spark_df=spark.read.option('header', 'true').csv('dataset.csv', inferSchema=True)

In [5]:
spark_df.head()

Row(Company='Google', Symbol='A1', Date=datetime.date(2009, 2, 1), Close=12.97, High=13.04, Low=12.97, Open=12.98, Volume=385666, Yearly=254151, Containings=254151)

In [6]:
spark_df.show()

+-------+------+----------+-----+-----+-----+-----+------+------+-----------+
|Company|Symbol|      Date|Close| High|  Low| Open|Volume|Yearly|Containings|
+-------+------+----------+-----+-----+-----+-----+------+------+-----------+
| Google|    A1|2009-02-01|12.97|13.04|12.97|12.98|385666|254151|     254151|
| Google|    A2|2009-02-02| 12.4|13.08| 12.4|13.08|385667|254152|     254152|
| Google|    A3|2009-02-03|12.72|12.93|12.72| 12.8|385668|254153|     254153|
| Google|    A4|2009-02-04|12.56| 12.8|12.56|12.59|385669|254154|     254154|
| Google|    A5|2009-02-05| 11.9|12.67| 11.9|12.43|385670|254155|     254155|
| Google|    A6|2009-02-06|11.85|12.21|11.85|12.12|385671|254156|     254156|
| Google|    A7|2009-02-07|12.13|12.13|11.91| 12.1|385672|254157|     254157|
| Google|    A8|2009-02-08|11.61|11.91|11.61|11.85|385673|254158|     254158|
| Google|    A9|2009-02-09|10.49|10.84|10.49|10.73|385674|254159|     254159|
| Google|   A10|2009-02-10| 11.1| 11.1|10.63|10.63|385675|254160

In [7]:
spark_df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Yearly: integer (nullable = true)
 |-- Containings: integer (nullable = true)



In [8]:
spark_df=spark_df.drop("Yearly","Containings")

In [9]:
spark_df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [10]:
spark_df.dtypes

[('Company', 'string'),
 ('Symbol', 'string'),
 ('Date', 'date'),
 ('Close', 'double'),
 ('High', 'double'),
 ('Low', 'double'),
 ('Open', 'double'),
 ('Volume', 'int')]

In [11]:
spark_df.na.drop().show()

+-------+------+----------+-----+-----+-----+-----+------+
|Company|Symbol|      Date|Close| High|  Low| Open|Volume|
+-------+------+----------+-----+-----+-----+-----+------+
| Google|    A1|2009-02-01|12.97|13.04|12.97|12.98|385666|
| Google|    A2|2009-02-02| 12.4|13.08| 12.4|13.08|385667|
| Google|    A3|2009-02-03|12.72|12.93|12.72| 12.8|385668|
| Google|    A4|2009-02-04|12.56| 12.8|12.56|12.59|385669|
| Google|    A5|2009-02-05| 11.9|12.67| 11.9|12.43|385670|
| Google|    A6|2009-02-06|11.85|12.21|11.85|12.12|385671|
| Google|    A7|2009-02-07|12.13|12.13|11.91| 12.1|385672|
| Google|    A8|2009-02-08|11.61|11.91|11.61|11.85|385673|
| Google|    A9|2009-02-09|10.49|10.84|10.49|10.73|385674|
| Google|   A10|2009-02-10| 11.1| 11.1|10.63|10.63|385675|
| Google|   A11|2009-02-11|10.25|10.81|10.25|10.81|385676|
| Google|   A12|2009-02-12| 8.61| 9.08| 8.61| 9.08|385677|
| Google|   A13|2009-02-13| 8.66| 8.81| 8.66| 8.81|385678|
| Google|   A14|2009-02-14| 9.35| 9.35| 8.74| 8.74|38567

In [12]:
spark_df.filter("Close=0").show()

+-------+------+----+-----+----+---+----+------+
|Company|Symbol|Date|Close|High|Low|Open|Volume|
+-------+------+----+-----+----+---+----+------+
+-------+------+----+-----+----+---+----+------+



In [13]:
spark_df = spark_df.withColumn("Close", spark_df["Close"].cast("float")) \
    .withColumn("High", spark_df["High"].cast("float")) \
    .withColumn("Low", spark_df["Low"].cast("float")) \
    .withColumn("Open", spark_df["Open"].cast("float"))

In [14]:
spark_df.dtypes

[('Company', 'string'),
 ('Symbol', 'string'),
 ('Date', 'date'),
 ('Close', 'float'),
 ('High', 'float'),
 ('Low', 'float'),
 ('Open', 'float'),
 ('Volume', 'int')]

## Perform statistical analysis on the stock market data:

In [15]:
grouped_data = spark_df.groupBy("Company")

In [16]:
grouped_data

In [17]:
average_closing_price = grouped_data.agg(avg("Close").alias("AvgClosingPrice"))

In [18]:
average_closing_price.show()

+--------+------------------+
| Company|   AvgClosingPrice|
+--------+------------------+
| Infinix|2.9094599962234495|
|   Nokia| 8.017189965248107|
|    One+|2.3719799971580504|
| Telpark| 1.712940002679825|
|Motorola|  2.99792999625206|
|  Realme| 4.324689993858337|
|    Nasa| 2.020989979505539|
|   Redmi|            19.195|
|    Poco|3.2646500134468077|
|    Oppo|21.193299961090087|
|  Techno|18.633450021743773|
|   Huwai|   2.2851700091362|
| Samsung|19.934100017547607|
|     HTC|0.5222700017690659|
|  Google|14.565151522857974|
|    Vivo|0.6193199977278709|
|   Apple|19.739599924087525|
|Q-Mobile| 3.737999997138977|
|    Deny|3.3789899921417237|
|     TCL|0.5106799983978272|
+--------+------------------+



In [19]:
highest_trading_volume =grouped_data.max("Volume")
highest_trading_volume = highest_trading_volume.orderBy(desc("max(Volume)")).limit(5).alias("MaxVolume")

In [20]:
highest_trading_volume.show()

+--------+-----------+
| Company|max(Volume)|
+--------+-----------+
|    Vivo|    8140000|
|Motorola|    7240000|
|     HTC|    3270000|
| Telpark|    3150000|
|    One+|    3010000|
+--------+-----------+



In [21]:
spark_df.show()

+-------+------+----------+-----+-----+-----+-----+------+
|Company|Symbol|      Date|Close| High|  Low| Open|Volume|
+-------+------+----------+-----+-----+-----+-----+------+
| Google|    A1|2009-02-01|12.97|13.04|12.97|12.98|385666|
| Google|    A2|2009-02-02| 12.4|13.08| 12.4|13.08|385667|
| Google|    A3|2009-02-03|12.72|12.93|12.72| 12.8|385668|
| Google|    A4|2009-02-04|12.56| 12.8|12.56|12.59|385669|
| Google|    A5|2009-02-05| 11.9|12.67| 11.9|12.43|385670|
| Google|    A6|2009-02-06|11.85|12.21|11.85|12.12|385671|
| Google|    A7|2009-02-07|12.13|12.13|11.91| 12.1|385672|
| Google|    A8|2009-02-08|11.61|11.91|11.61|11.85|385673|
| Google|    A9|2009-02-09|10.49|10.84|10.49|10.73|385674|
| Google|   A10|2009-02-10| 11.1| 11.1|10.63|10.63|385675|
| Google|   A11|2009-02-11|10.25|10.81|10.25|10.81|385676|
| Google|   A12|2009-02-12| 8.61| 9.08| 8.61| 9.08|385677|
| Google|   A13|2009-02-13| 8.66| 8.81| 8.66| 8.81|385678|
| Google|   A14|2009-02-14| 9.35| 9.35| 8.74| 8.74|38567

In [22]:
closing_price = spark_df.select('Company', 'Close').groupBy('Company')


In [23]:
sum_closing_price = grouped_data.agg(sum('Close').alias('Total_Close'))

In [24]:
sum_closing_price.show()

+--------+------------------+
| Company|       Total_Close|
+--------+------------------+
| Infinix|290.94599962234497|
|   Nokia| 801.7189965248108|
|    One+|237.19799971580505|
| Telpark|171.29400026798248|
|Motorola|  299.792999625206|
|  Realme|432.46899938583374|
|    Nasa| 202.0989979505539|
|   Redmi|            1919.5|
|    Poco| 326.4650013446808|
|    Oppo| 2119.329996109009|
|  Techno|1863.3450021743774|
|   Huwai|   228.51700091362|
| Samsung|1993.4100017547607|
|     HTC|52.227000176906586|
|  Google|1441.9500007629395|
|    Vivo|61.931999772787094|
|   Apple|1973.9599924087524|
|Q-Mobile| 373.7999997138977|
|    Deny|337.89899921417236|
|     TCL|51.067999839782715|
+--------+------------------+



In [25]:
print(closing_price)

In [34]:
df_percent = spark_df.withColumn('Close_price_percent', col('Close')/sum('Close').over(Window.partitionBy('Company'))*100)
df_cum_percent_grp = df_percent.withColumn('Close_price_percentage', sum(df_percent.Close_price_percent).over(Window.partitionBy('Company').rowsBetween(-sys.maxsize, 0)))
df_cum_percent_grp=df_cum_percent_grp.drop('Close_price_percent')
df_cum_percent_grp.show()


+-------+------+----------+-----+-----+-----+-----+------+----------------------+
|Company|Symbol|      Date|Close| High|  Low| Open|Volume|Close_price_percentage|
+-------+------+----------+-----+-----+-----+-----+------+----------------------+
|  Apple|  A200|2009-02-01|23.98|24.06|23.34|23.82|385865|    1.2148168977312654|
|  Apple|  A201|2009-02-02|23.85|24.34|23.59|24.19|385866|     2.423048091534055|
|  Apple|  A202|2009-02-03|23.31|24.11|23.17|23.92|385867|    3.6039230614212627|
|  Apple|  A203|2009-02-04|23.51|23.62|22.98|23.38|385868|     4.794929987564352|
|  Apple|  A204|2009-02-05|23.47|23.74|23.39|23.51|385869|     5.983910483806066|
|  Apple|  A205|2009-02-06| 24.0|24.09|23.44|23.63|385870|     7.199740596488018|
|  Apple|  A206|2009-02-07|23.61|24.11|23.49|24.02|385871|     8.395813500759047|
|  Apple|  A207|2009-02-08|23.78|23.82|23.42|23.65|385872|     9.600498538859927|
|  Apple|  A208|2009-02-09|23.17|23.76|22.58|23.76|385873|    10.774281197343315|
|  Apple|  A209|

In [36]:
max_change_day = df_cum_percent_grp.orderBy(desc('Close_price_percentage')).first()
max_change_day

Row(Company='Nokia', Symbol='A699', Date=datetime.date(2009, 5, 11), Close=4.275000095367432, High=4.4720001220703125, Low=4.269999980926514, Open=4.410999774932861, Volume=281710, Close_price_percentage=100.00000000000007)

In [38]:
min_change_day = df_cum_percent_grp.orderBy('Close_price_percentage').first()
min_change_day

Row(Company='Deny', Symbol='A1200', Date=datetime.date(2009, 2, 1), Close=2.2279999256134033, High=2.2920000553131104, Low=2.2279999256134033, Open=2.2920000553131104, Volume=115440, Close_price_percentage=0.6593686074226038)

In [44]:

print("Day with highest percentage change:", max_change_day['Date'], max_change_day['Company'], max_change_day['Close_price_percentage'])
# print('_____________________________________')

print('Day with lowest percentage change:', min_change_day['Date'], min_change_day['Company'], min_change_day['Close_price_percentage'])

Day with highest percentage change: 2009-05-11 Nokia 100.00000000000007
Day with lowest percentage change: 2009-02-01 Deny 0.6593686074226038


## Perform time series analysis on the stock market data: